# Utilidades de carga de datos JSON

In [ ]:
"""Utilidades para cargar datos JSON desde `data/raw`.

Este módulo ofrece funciones para:
- Listar todos los archivos `.json` existentes en un directorio.
- Cargar el contenido de esos `.json` en memoria.
"""

from __future__ import annotations

import json
from pathlib import Path
from typing import Any, Dict, List, Union


def listar_archivos_json(directorio: str | Path = "data/raw", recursivo: bool = False) -> List[Path]:
    """Devuelve la lista de archivos `.json` en el directorio dado.

    Args:
        directorio: Ruta del directorio a inspeccionar.
        recursivo: Si es True, busca también en subdirectorios.

    Returns:
        Lista de rutas (`Path`) a archivos `.json`.
    """
    base = Path(directorio)
    if not base.exists() or not base.is_dir():
        raise FileNotFoundError(f"No se encontró el directorio: {base}")

    patron = "**/*.json" if recursivo else "*.json"
    return [p for p in sorted(base.glob(patron)) if p.is_file()]


def cargar_todos_los_json(rutas: List[str | Path], estricto: bool = True) -> List[Any]:
    """Carga todos los archivos `.json` de un conjunto de rutas.

    Lee cada archivo `.json` provisto en `rutas` y devuelve una lista con
    el contenido parseado de cada uno, en el mismo orden de entrada.

    Args:
        rutas: Lista o conjunto de rutas a archivos `.json`.
        estricto: Si True, lanza excepción ante JSON inválido; de lo
                  contrario, ignora archivos con errores y continúa.

    Returns:
        Lista con el contenido JSON de cada archivo.

    Nota:
        No se devuelve un `set` porque los objetos JSON (dict/list) no son
        hashables. Si necesitas eliminar duplicados, puedes convertir cada
        elemento a cadena con `json.dumps(..., sort_keys=True)` y operar allí.
    """
    resultados: List[Any] = []
    for r in rutas:
        ruta = Path(r)
        if not ruta.exists() or not ruta.is_file():
            raise FileNotFoundError(f"No existe el archivo: {ruta}")
        if ruta.suffix.lower() != ".json":
            continue
        try:
            with ruta.open("r", encoding="utf-8") as f:
                datos = json.load(f)
            resultados.append(datos)
        except json.JSONDecodeError as e:
            if estricto:
                raise ValueError(f"JSON inválido en {ruta}: {e}") from e
            # En modo no estricto, omitimos archivos con error
    return resultados


def extrae_html(
    datos_json: Any,
    clave: str = "html",
    omitir_vacios: bool = True,
) -> List[Dict[str, Any]]:
    """Extrae solo el campo HTML de una estructura JSON agrupada por archivo.

    Acepta estructuras como:
        { "archivo.json": [ {"url":..., "html":..., ...}, ... ] }
    y devuelve:
        [ {"html": ...}, {"html": ...}, ... ]

    También funciona si `datos_json` es una lista (o lista de listas) de
    registros tipo dict. Recorre recursivamente y extrae cualquier dict que
    contenga la clave indicada.

    Args:
        datos_json: Estructura con registros que incluyen la clave de HTML.
        clave: Nombre de la clave a extraer (por defecto "html").
        omitir_vacios: Si True, omite valores vacíos o None.

    Returns:
        Lista de diccionarios con solo la clave de HTML.
    """
    resultados: List[Dict[str, Any]] = []

    def visitar(nodo: Any) -> None:
        if isinstance(nodo, dict):
            if clave in nodo:
                valor = nodo.get(clave)
                if not (
                    omitir_vacios
                    and (
                        valor is None
                        or (isinstance(valor, str) and valor.strip() == "")
                    )
                ):
                    resultados.append({clave: valor})
            for v in nodo.values():
                visitar(v)
        elif isinstance(nodo, list):
            for elem in nodo:
                visitar(elem)
        # otros tipos se ignoran

    visitar(datos_json)
    return resultados




def _marker_variants(m: str) -> List[str]:
    """Genera variantes del marcador con y sin escapes de comillas.

    Incluye:
    - El marcador tal cual.
    - Versión con `\"` reemplazado por `"`.
    - Versión con `"` reemplazado por `\"`.
    - Versión sin espacios extremos.
    """
    return list({
        m,
        m.strip(),
        m.replace(r'\"', '"'),
        m.replace('"', r'\"'),
    })


def extrae_texto_json_entre(
    file_json: str | Path,
    inicio: str = '"items":[',
    fin: str = '],"totalPages"',
) -> str:
    """Extrae el texto entre dos marcadores dentro de un archivo JSON.

    Lee el archivo como texto y devuelve el substring comprendido entre la
    primera ocurrencia de `inicio` y la siguiente ocurrencia de `fin`.

    Por compatibilidad, intenta variantes con comillas escapadas (\\") si
    no encuentra coincidencia exacta.

    Args:
        file_json: Ruta al archivo `.json` a leer.
        inicio: Marcador inicial. Por defecto busca '"items":['.
        fin: Marcador final. Por defecto busca '],"totalPages"'.

    Returns:
        El texto encontrado entre `inicio` y `fin`. Si no se encuentran
        los marcadores en ninguna variante, devuelve cadena vacía.
    """
    ruta = Path(file_json)
    if not ruta.exists() or not ruta.is_file():
        raise FileNotFoundError(f"No existe el archivo: {ruta}")

    contenido = ruta.read_text(encoding="utf-8", errors="ignore")

    inicio_real: str | None = None
    for variante in _marker_variants(inicio):
        idx = contenido.find(variante)
        if idx != -1:
            inicio_real = variante
            inicio_idx = idx + len(variante)
            break

    if inicio_real is None:
        return ""

    fin_real: str | None = None
    for variante in _marker_variants(fin):
        fin_idx = contenido.find(variante, inicio_idx)
        if fin_idx != -1:
            fin_real = variante
            break

    if fin_real is None:
        return ""

    return contenido[inicio_idx:fin_idx]


def main() -> None:
    """Ejecución ad-hoc: carga JSON por rutas y extrae `items`."""
    try:
        archivos = listar_archivos_json("data/raw", recursivo=False)
    except FileNotFoundError as e:
        print(e)
        return

    print(f"Encontrados {len(archivos)} archivo(s) .json en data/raw")
    contenidos = cargar_todos_los_json(archivos, estricto=False)
    print(f"Cargados {len(contenidos)} archivo(s) JSON")


if __name__ == "__main__":
    main()


In [52]:
# Redefinición: extrae_texto_json_entre para trabajar sobre contenido_html
def extrae_texto_json_entre(
    contenido_html: Any,
    ini_text: str ='"items":[{"bodyTypeId":',
    fin_text: str = '}],"totalPages"'
) -> List[str]:
    """Extrae, para cada HTML, el bloque entre `inicio` y `fin` y reconstruye \"items\": [...].

    Entrada admitida: lista de dicts con clave 'html', lista de strings o string único.
    Del marcador final elimina ,\"totalPages\" (o su versión escapada) para formar "items": [...].
    """
    textos: List[str] = []
    if isinstance(contenido_html, str):
        textos = [contenido_html]
    elif isinstance(contenido_html, list):
        for elem in contenido_html:
            if isinstance(elem, dict) and 'html' in elem and isinstance(elem['html'], str):
                textos.append(elem['html'])
            elif isinstance(elem, str):
                textos.append(elem)
            elif isinstance(elem, list):
                for sub in elem:
                    if isinstance(sub, dict) and 'html' in sub and isinstance(sub['html'], str):
                        textos.append(sub['html'])
                    elif isinstance(sub, str):
                        textos.append(sub)
    elif isinstance(contenido_html, dict) and 'html' in contenido_html and isinstance(contenido_html['html'], str):
        textos = [contenido_html['html']]

    resultados: List[str] = []
    for contenido in textos:
        contenido = contenido.replace("\\", "")  
        ini = contenido.find(ini_text)
        if ini == -1:
            return ""
        ini += len(ini_text)
        fin = contenido.find(fin_text, ini)
        if fin == -1:
            return ""
        cuerpo = contenido[ini:fin]
        fin_text= fin_text.replace(',\"totalPages\"', '')
        fragmento = f"{ini_text}{cuerpo}{fin_text}"
        resultados.append(fragmento)
    return resultados


In [46]:
def extraer_entre(texto,
                 ini_text: str ='"items":[{"bodyTypeId":',
                 fin_text: str = '}],"totalPages"',
                ):
    """Devuelve el texto entre ini_text y fin_text. Si no se encuentra, devuelve una cadena vacía."""
    ini = texto.find(ini_text)
    if ini == -1:
        return ""
    ini += len(ini_text)
    fin = texto.find(fin_text, ini)
    if fin == -1:
        return ""
    cuerpo = texto[ini:fin]
    fin_text= fin_text.replace(',\"totalPages\"', '')
    fragmento = f"{ini_text}{cuerpo}{fin_text}"

    return fragmento




In [57]:
archivos = listar_archivos_json("data/raw", recursivo=False)
contenidos = cargar_todos_los_json(archivos, estricto=False)
contenido_html = extrae_html(contenidos)
extrae_items = extrae_texto_json_entre(contenido_html)

In [58]:
extrae_items                           

['"items":[{"bodyTypeId":14,"creationDate":"2025-08-26T09:57:53Z","hp":110,"fuelType":"Gasolina","fuelTypeId":2,"hasWarranty":true,"id":"8762366","img":"https://a.mcdn.es/mnet/2025/08/26/8762366/32372272_g.jpg","imgUrl":"https://a.mcdn.es/mnet/2025/08/26/8762366/32372272_g.jpg/359x269cut/","isCertified":false,"isFinanced":false,"isUrgent":false,"isProfessional":true,"isUrlSemantic":true,"km":8000,"location":{"provinceIds":[20],"regionId":18,"regionLiteral":"Pau00EDs Vasco","mainProvince":"Guipu00FAzcoa","mainProvinceId":20,"cityId":20903,"cityLiteral":"Astigarraga"},"make":"BMW","makeId":7,"modelId":5326,"pack":{"legacyId":18,"type":"start"},"price":9995,"url":"/ocasion/bmw/r_ninet/scrambler-2018-en-guipuzcoa-8762366.htm","offerType":{"id":0,"literal":"Ocasiu00F3n"},"phone":"943470919","photos":["https://a.mcdn.es/mnet/2025/08/26/8762366/32372272_g.jpg","https://a.mcdn.es/mnet/2025/08/26/8762366/32372273_g.jpg","https://a.mcdn.es/mnet/2025/08/26/8762366/32372274_g.jpg","https://a.mcdn.

In [68]:
import json
from typing import List, Union

BASE_URL = "https://motos.coches.net/"

def _find_json_array_after_items(s: str) -> str:
    key = '"items":['
    start = s.find(key)
    if start == -1:
        return ""
    i = start + len(key) - 1
    depth = 0
    arr_start = None
    for pos in range(i, len(s)):
        ch = s[pos]
        if ch == '[':
            if depth == 0:
                arr_start = pos
            depth += 1
        elif ch == ']':
            depth -= 1
            if depth == 0 and arr_start is not None:
                return s[arr_start:pos+1]
    return ""

def _normalize_url(url: str) -> str:
    if not url:
        return url
    u = url.strip()
    if u.startswith("http://") or u.startswith("https://"):
        return u
    # Evita doble barra al unir
    if u.startswith("/"):
        u = u[1:]
    return BASE_URL + u

def extra_parse_item(extrae_items: Union[str, List[str]], extrac_list: List[str] = None) -> List[dict]:
    if extrac_list is None:
        extrac_list = ['km', 'precio', 'year']

    norm_map = {
        'km': 'km',
        'precio': 'price',
        'price': 'price',
        'year': 'year',
        'año': 'year',
        'anio': 'year',
        'url': 'url',
    }
    wanted = [norm_map.get(f.lower(), f) for f in extrac_list]

    chunks = [extrae_items] if isinstance(extrae_items, str) else list(extrae_items)
    resultados = []

    for chunk in chunks:
        if not isinstance(chunk, str):
            continue
        arr_text = _find_json_array_after_items(chunk)
        if not arr_text:
            continue

        try:
            items = json.loads(arr_text)
            if not isinstance(items, list):
                continue
        except json.JSONDecodeError:
            continue

        for obj in items:
            if not isinstance(obj, dict):
                continue
            row = {}

            # siempre útil añadir id si está
            if 'id' in obj:
                row['id'] = obj.get('id')

            # añade url siempre que exista, normalizada
            if 'url' in obj:
                row['url'] = _normalize_url(obj.get('url'))

            for f in wanted:
                val = obj.get(f, None)

                if f == 'url':
                    row['url'] = _normalize_url(val)
                    continue

                if isinstance(val, str) and f in ('km', 'price', 'year'):
                    num = ''.join(ch for ch in val if ch.isdigit())
                    val = int(num) if num else None

                row[f] = val

            resultados.append(row)

    return resultados


In [73]:
res = extra_parse_item(extrae_items, extrac_list=['title','km', 'price', 'year','url','imgUrl','mainProvince'])

In [74]:
res


[{'id': '8762366',
  'url': 'https://motos.coches.net/ocasion/bmw/r_ninet/scrambler-2018-en-guipuzcoa-8762366.htm',
  'title': 'BMW R nineT Scrambler',
  'km': 8000,
  'price': 9995,
  'year': 2018,
  'imgUrl': 'https://a.mcdn.es/mnet/2025/08/26/8762366/32372272_g.jpg/359x269cut/',
  'mainProvince': None},
 {'id': '8760435',
  'url': 'https://motos.coches.net/ocasion/bmw/r_ninet//5-2019-en-madrid-8760435.htm',
  'title': 'BMW R nineT /5',
  'km': 12960,
  'price': 11990,
  'year': 2019,
  'imgUrl': 'https://a.mcdn.es/mnet/2025/08/23/8760435/32358302_g.jpg/359x269cut/',
  'mainProvince': None},
 {'id': '8760378',
  'url': 'https://motos.coches.net/ocasion/bmw/r_ninet/2014-en-murcia-8760378.htm',
  'title': 'BMW R nineT',
  'km': 22500,
  'price': 9500,
  'year': 2014,
  'imgUrl': 'https://a.mcdn.es/mnet/2025/08/23/8760378/32356808_g.jpg/359x269cut/',
  'mainProvince': None},
 {'id': '8759793',
  'url': 'https://motos.coches.net/ocasion/bmw/r_ninet/2014-en-madrid-8759793.htm',
  'title':

In [ ]:
 {\"bodyTypeId\":14,
\"creationDate\":\"2025-01-22T12:56:21Z\",
\"hp\":110,
\"fuelType\":\"Gasolina\",
\"fuelTypeId\":2,
\"hasWarranty\":true,
\"id\":\"8576019\",
\"img\":\"https://a.mcdn.es/mnet/2025/01/22/8576019/30562965_g.jpg\",
\"imgUrl\":\"https://a.mcdn.es/mnet/2025/01/22/8576019/30562965_g.jpg/359x269cut/\",
\"isCertified\":false,
\"isFinanced\":true,
\"isUrgent\":false,
\"isProfessional\":true,
\"isUrlSemantic\":true,
\"km\":15000,
\"location\":{\"provinceIds\":[24],
\"regionId\":7,
\"regionLiteral\":\"Castilla y Le\u00F3n\",
\"mainProvince\":\"Le\u00F3n\",
\"mainProvinceId\":24,
\"cityId\":24142,
\"cityLiteral\":\"San Andr\u00E9s del Rabanedo\"},
\"make\":\"BMW\",\"makeId\":7,\"modelId\":5326,
\"pack\":{\"legacyId\":21,
\"type\":\"advance\"},
\"price\":10500,
\"url\":\"/ocasion/bmw/r_ninet/scrambler-2016-en-leon-8576019.htm\",
\"offerType\":{\"id\":0,
\"literal\":\"Ocasi\u00F3n\"},
\"phone\":\"987981024\",
\"provinceId\":24,
\"publicationDate\":\"2025-10-19T11:33:53Z\",
\"specificFuelTypeId\":2,
\"title\":\"BMW R nineT Scrambler\",
\"videos\":[],
\"warrantyText\":\"1 A\u00D1O\",
\"year\":2016,
\"seller\":{\"name\":\"Bmw Bernesga Motor \",
\"isProfessional\":true,
\"contractId\":\"1001564\",
\"pack\":{\"legacyId\":21,
\"type\":\"advance\"}}},
